In [1]:
import sys
sys.path.append('../') # add this project path to the system path to import own module

from Util.util import Constant, WaitValueMatch
from Util import util
import time
import requests
import pickle
from Ip.ipCheck import IpPool, Ip
from Account.rawAccountPool import raw_accounts
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# option = webdriver.ChromeOptions()
# option.add_experimental_option("detach", True)
# driver = webdriver.Chrome(chrome_options=option)

In [2]:
class AccountPool(object):
    def __init__(self):
        self.raw_accounts = raw_accounts
        self.ipPool = self.getCurrentIpPool()
        self.accountPool = self.setAccountPool()
        self.saveIpAccountPool()

    def __getitem__(self, index):
        return self.accountPool[index]
    
    def __len__(self):
        return len(self.accountPool)

    def saveIpAccountPool(self): # save the array into a file called ipPool.pkl
        with open(Constant.ip_pool_dir, 'wb') as fout:
            pickle.dump(self.ipPool, fout, pickle.HIGHEST_PROTOCOL)
        
        with open(Constant.account_pool_dir, 'wb') as fout:
            pickle.dump(self.accountPool, fout, pickle.HIGHEST_PROTOCOL)

    def setAccountPool(self):
        ip_len = len(self.ipPool)
        acc_len = len(raw_accounts)
        print('account len: ' + str(acc_len))
        print('ip len: ' + str(ip_len))
        acc_ind = 0
        accountPool = []
        for ip_ind in range(0, ip_len):
            if(acc_ind >= acc_len):
                break
            if(not self.ipPool[ip_ind].is_alive):
                continue
            if(self.ipPool[ip_ind].is_used):
                continue
            self.ipPool[ip_ind].is_used = True
            accountPool.append(Account(self.ipPool[ip_ind], raw_accounts[acc_ind]))
            acc_ind += 1
        return accountPool
    
    def getCurrentIpPool(self): # get real-time ip pool (cost time to test all ip addr if alive or not)
        return IpPool()

    def getSavedIpPool(self): # get ip pool from a file (stored before)
        with open(Constant.ip_pool_dir, 'rb') as fin: 
            return pickle.load(fin)

class Account(object):
    def __init__(self, ipObj, raw_account):
        self.ipObj = ipObj
        self.buff_phone = raw_account['buff_phone']
        self.buff_pwd = raw_account['buff_pwd']
        self.steam_email = raw_account['steam_email']
        self.steam_account = raw_account['steam_account']
        self.steam_pwd = raw_account['steam_pwd']
        self.login_time = 0
        self.cookie, self.csrf_token = self.setCookieAndToken()
        self.is_login = self.isLogin()
        self.is_market = self.isMarketAvailable()
    
    def setCookieAndToken(self): # process cookie map to cookie str and token str
        cookie_map = self.getCookie()
        cookie = ''
        csrf_token = ''
        for _map in cookie_map:
            if(_map['name'] == 'csrf_token'):
                csrf_token = _map['value']
            else:
                cookie += _map['name'] + '=' + _map['value'] + '; '
        return cookie, csrf_token
    
    def getCookie(self): # get cookies stored in the chrome
        proxy = Proxy({
            'proxyType': ProxyType.MANUAL,
            'httpProxy': self.ipObj.ip,
            'sslProxy': self.ipObj.ip,
            'noProxy': ''
        })

        options =  webdriver.ChromeOptions()
        options.proxy = proxy
        options.add_experimental_option("detach", True) # make the chrome window always opened
        time.sleep(2)
        driver = webdriver.Chrome(options = options)
        
        
        driver.get(Constant.domain)

        driver.find_element_by_xpath('//div[@class="nav nav_entries"]/ul/li/a[@onclick="loginModule.showLogin()"]').click() # click login/register button
        driver.execute_script("arguments[0].setAttribute('class', 'on')", driver.find_element_by_xpath('//div[@id="remember-me"]/span[@value="y"]')) # agree signed for 10 days
        driver.execute_script("arguments[0].setAttribute('value', 'y')", driver.find_element_by_xpath('//div[@id="remember-me"]'))
        driver.execute_script("arguments[0].setAttribute('class', 'on')", driver.find_element_by_xpath('//div[@id="agree-checkbox"]/span[@value="agree"]')) # agree login agreement
        driver.execute_script("arguments[0].setAttribute('value', 'agree')", driver.find_element_by_xpath('//div[@id="agree-checkbox"]'))

        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element_by_xpath('//div[@id="j_login"]/iframe'))) # swith to login iframe
        
        driver.find_element_by_id('phoneipt').send_keys(self.buff_phone) # input phone number
        if self.buff_pwd != '': # if has buff password
            driver.find_element_by_xpath('//a[@class="tab0"]').click()
            driver.find_element_by_xpath('//input[@class="j-inputtext dlemail"]').send_keys(self.buff_pwd)
            time.sleep(5)
            driver.find_element_by_id('submitBtn').click()
        else:
            WebDriverWait(driver, 100).until(WaitValueMatch((By.CLASS_NAME, 'j-inputtext'), r'[0-9]+'))
            driver.find_element_by_class_name('u-loginbtn btncolor tabfocus').click()
        driver.switch_to.default_content()
        WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, 'go_to_steam')))
        
        self.login_time = time.time()
        return driver.get_cookies()
    

    def isMarketAvailable(self): # check if this account can access market
        if self.is_login == False:
            return False
        return self.checkLoginAndMarket(False)
    
    def isLogin(self): # check if this account login
        return self.checkLoginAndMarket(True)
    
    def checkLoginAndMarket(self, login): # check if this account login or can access market
        resp = requests.get(Constant.account_availability_check_addr, self.ipObj.proxies)
        code = resp.json()['code']
        print(code)
        if code == 'ok':
            return True
        elif login:
            if code == 'Login Required':
                return False
            else:

                return True
        else:
            if code == 'Action Forbidden':
                return False
            else:
                return True

In [3]:
AccountPool()

200
200
200
200
account len: 6
ip len: 4
Login Required
Login Required
Login Required
Login Required


In [ ]:
[{'domain': 'buff.163.com', 'httpOnly': True, 'name': 'session', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '1-T3uY3lAqWbt2_jkteUwqUtBCzjdOconOcNucoeY5mPd32033790545'}, {'domain': '.163.com', 'expiry': 1713165768, 'httpOnly': False, 'name': 'P_INFO', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '17056635880|1681629766|1|netease_buff|00&99|zhj&1681629738&netease_buff#zhj&330200#10#0#0|&0|null|17056635880'}, {'domain': 'buff.163.com', 'httpOnly': False, 'name': 'csrf_token', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'ImNiZGY4NzFmNzA5YTlhZGRhMGRiMGNjZTBmNGIxODg3NzdiYTEwNmYi.Fx0zxg.VZSaYA4hXB2b9kNkG5RceLmT-Rc'}, {'domain': '.163.com', 'httpOnly': False, 'name': 'S_INFO', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '1681629766|0|0&60##|17056635880'}, {'domain': '.163.com', 'httpOnly': True, 'name': 'NTES_YD_SESS', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '92LhlYIX3sySG7X6ii7WJMwUeraRkpJd96j5AM1sffgX_q13_H8IY.1uOcVXrKFOXDb6okBjRk_OSvoU.BEUvhzSbxSLG2br30jZ_hA1DsBQ_eBRnJU96wil6ixrd8MTSv4EF.gNjlGl.xAMPMB2QxTzCQeCSv.XpEWUwGrYNYY7QAHTcN9OWeGkceBHeXrboi4Abj5nY2apGTQzlM2XOaEsKZVIsFkH88VqmRBWYHzLT'}, {'domain': 'buff.163.com', 'httpOnly': False, 'name': 'game', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'csgo'}, {'domain': 'buff.163.com', 'expiry': 1682522569, 'httpOnly': True, 'name': 'remember_me', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'U1106751753|7FCRoqQSn5lgTc2TsdxGcL22cACu4ppJ'}, {'domain': 'buff.163.com', 'httpOnly': False, 'name': 'Locale-Supported', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'en'}, {'domain': 'buff.163.com', 'expiry': 1716189761, 'httpOnly': False, 'name': 'Device-Id', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'r1bEXmZHUNpshcxQaO5i'}]

In [ ]:
Device-Id=iXGL41yiCcG12JnLmxRv; Locale-Supported=en; game=csgo; NTES_YD_SESS=eOIcvWwOu9sy5qnKU2W0PA9LU44yLiuivsNTu.GSqqxtd3GbdnHABwGMUf8t07gUtCYs_jZNojdUkR_pwZPpRVQkYOkEvhY0bKNWdVuGCSZFdDZorIpeszX2sXO0yH.4kRcPgwxaN2v2wOu.9.ZhFO4QiFDikRwtLPmpzv0BaBBa9RRNz4.VHIknOtKjMpDe4SqtNITqZ1iJQ.gtEDQqFzPS7W8ASgjnHH836oZmBnQE4; S_INFO=1681651004|0|0&60##|17056635880; P_INFO=17056635880|1681651004|1|netease_buff|00&99|zhj&1681649699&netease_buff#zhj&330200#10#0#0|&0|null|17056635880; remember_me=U1106751753|kSW334dodkb3tB68UNf5YoRgoGZc0Ovs; session=1-Y4UAYPQ4HG6wyZhaPu7IcC4CvGaRIkaYXJyrPa59D_ON2033790545; csrf_token=ImNjOWQ5ZGY5YWVhOTMxMGE2NzRhYTY2MjM0OWQ0ODQxNzg0YjZiMjAi.Fx2GvA.53d0gtn9zzMJHrUJmATcVmZjlJ8
Device-Id=iXGL41yiCcG12JnLmxRv; Locale-Supported=en; game=csgo; NTES_YD_SESS=eOIcvWwOu9sy5qnKU2W0PA9LU44yLiuivsNTu.GSqqxtd3GbdnHABwGMUf8t07gUtCYs_jZNojdUkR_pwZPpRVQkYOkEvhY0bKNWdVuGCSZFdDZorIpeszX2sXO0yH.4kRcPgwxaN2v2wOu.9.ZhFO4QiFDikRwtLPmpzv0BaBBa9RRNz4.VHIknOtKjMpDe4SqtNITqZ1iJQ.gtEDQqFzPS7W8ASgjnHH836oZmBnQE4; S_INFO=1681651004|0|0&60##|17056635880; P_INFO=17056635880|1681651004|1|netease_buff|00&99|zhj&1681649699&netease_buff#zhj&330200#10#0#0|&0|null|17056635880; remember_me=U1106751753|kSW334dodkb3tB68UNf5YoRgoGZc0Ovs; session=1-Y4UAYPQ4HG6wyZhaPu7IcC4CvGaRIkaYXJyrPa59D_ON2033790545; csrf_token=ImNjOWQ5ZGY5YWVhOTMxMGE2NzRhYTY2MjM0OWQ0ODQxNzg0YjZiMjAi.Fx2Gxw.iRqZK5sgVNOVyhR6_760fQWzcts